In [1]:
import os, sys, glob
from keras import backend as K
import numpy as np
from shutil import copyfile

%matplotlib inline

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
homeDir = os.getcwd()
dataHome = homeDir + "/data/"
dataDir = homeDir + "/data/"
#dataDir = homeDir + "/data/sample/"
train_path = dataDir + "train/"
valid_path = dataDir + "valid/"
model_path = dataDir + "models/"
test_path = dataHome + "test/"
results_path = dataDir + "results/"

In [3]:
%cd $homeDir

/home/ubuntu/KaggleCompetitions/Fisheries


In [4]:
from utils import *
from Vgg16 import Vgg16
from vgg16bn import Vgg16BN

In [5]:
vgg = Vgg16()

In [6]:
#Set constants. You can experiment with no_of_epochs to improve the model. You can reduce the batch_size 
#depending on the memory contraints of gpu
batch_size=64
no_of_epochs=1

In [7]:
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)

Found 3342 images belonging to 8 classes.
Found 435 images belonging to 8 classes.


In [8]:
vgg.model.pop() #Remove the final layer
for layer in vgg.model.layers:
    layer.trainable=False # Set all other layers to untrainable

In [9]:
vgg.model.add(Dense(8, activation='softmax')) 
#Adding a new dense layer wiht only 8 outputs and softmax acitvation as it is the output layer

In [10]:
vgg.model.load_weights(results_path+'f1LastLayer.h5')

In [12]:
vgg.compile(lr = 0.0001)

In [13]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
3342/3342 [==============================] - 101s - loss: 1.0587 - acc: 0.6795 - val_loss: 0.5772 - val_acc: 0.7977


In [16]:
vgg.fit(batches, val_batches, nb_epoch=2)

Epoch 1/2
3342/3342 [==============================] - 99s - loss: 1.0242 - acc: 0.6900 - val_loss: 0.5403 - val_acc: 0.8092
Epoch 2/2
3342/3342 [==============================] - 94s - loss: 0.9594 - acc: 0.6954 - val_loss: 0.5455 - val_acc: 0.8046


In [19]:
vgg.compile(lr = 0.001)

In [20]:
vgg.fit(batches, val_batches, nb_epoch=2)

Epoch 1/2
3342/3342 [==============================] - 98s - loss: 1.0162 - acc: 0.6879 - val_loss: 0.4696 - val_acc: 0.8207
Epoch 2/2
3342/3342 [==============================] - 93s - loss: 0.9284 - acc: 0.7223 - val_loss: 0.5598 - val_acc: 0.8115


In [23]:
vgg.compile(lr = 0.0001)
vgg.fit(batches, val_batches, nb_epoch=2)

Epoch 1/2
3342/3342 [==============================] - 99s - loss: 0.7715 - acc: 0.7636 - val_loss: 0.3728 - val_acc: 0.8690
Epoch 2/2
3342/3342 [==============================] - 94s - loss: 0.6942 - acc: 0.7573 - val_loss: 0.4105 - val_acc: 0.8598


In [ ]:
tbatches, preds = vgg.test(test_path, batch_size = 64)
filenames = tbatches.filenames

Found 1000 images belonging to 1 classes.


In [22]:
preds = np.clip(preds,0.01,0.99)
files = [s.split('/')[-1] for s in filenames]
import pandas as pd
df = pd.DataFrame(preds, columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
df.insert(0,'image', files)
subFile = results_path + 'finetuningthebeforebestmodel3clip.csv'
df.to_csv(subFile, index=False)